In [1]:
import sys
sys.path.insert(0, "/home/jack/github/turtle-backtest")
print(sys.path)

['/home/jack/github/turtle-backtest', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages', '/home/jack/github/turtle-backtest']


In [ ]:
from backtesting import Backtest, Strategy
from datetime import datetime
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.io as pio

# from backtesting.test import GOOG
from turtle.service.data_update_service import DataUpdateService
from turtle.common.enums import TimeFrameUnit
import turtle
print(turtle)

In [ ]:
class SignalStrategy(Strategy):
    def init(self):
        self.i = -1
        pass

    def next(self):
        current_signal = self.data.buy_signal[-1]
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1]*0.985)
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_20[-1] :
            self.position.close()

data_updater = DataUpdateService(time_frame_unit=TimeFrameUnit.DAY)

ticker = 'AVGO'
start_date: datetime = datetime(year=2024, month=5, day=1)
end_date: datetime = datetime(year=2024, month=8, day=30)

count = data_updater.darvas_box_strategy.trading_signals_count(ticker,start_date, end_date,)
# print(f"{ticker} - {count} trades")
if count > 0:
    df = data_updater.darvas_box_strategy.df.copy()
    # df.info()
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    df = df.set_index(["hdate"])
    # bt = Backtest(GOOG, SignalStrategy, commission=.002, cash = 10000)
    bt = Backtest(df, SignalStrategy, cash = 10000)
    stats=bt.run()
    # print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean())
    # print(data_updater.darvas_box_strategy.df.iloc[130:140])

    if not stats._trades.empty:
        # print(stats._trades.empty)
        print(stats._trades["PnL"].mean(), stats._trades["Duration"].mean())


fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.update_layout(
    title=f"{ticker} Stock OHLC Data",
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,  # Hides the range slider below the chart
    width=2000,  # Set the width
    height=1000,   # Set the height
    xaxis={
        'tickformat': '%d-%m-%Y',  # Format to show only date
        'rangeslider': {'visible': True, 'thickness': 0.1},  # Add a thin range slider for zoom
        'type': 'date'  # Ensure correct axis scaling
    },
    yaxis={'fixedrange': False},
)

for i, trade in stats._trades.iterrows():
    entry_time = trade['EntryTime']
    exit_time = trade['ExitTime']
    entry_price = trade['EntryPrice']
    exit_price = trade['ExitPrice']
    pnl = trade['PnL']

    # Color the line green for winning trades, red for losing trades
    line_color = 'green' if pnl > 0 else 'red'

    # Add a line connecting the buy and sell points
    fig.add_trace(go.Scatter(
        x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='lines+markers',
        marker={'color': line_color, 'size': 10},
        line={'color': line_color, 'width': 2},
        name=f"Trade {i+1} (PnL: {pnl:.2f})"
    ))

# fig.show()
pio.renderers.default = 'browser'
pio.show(fig)

In [ ]:
class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        current_signal = self.data.buy_signal[-1]
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1])
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_10[-1] :
            self.position.close()

df_collect = pd.DataFrame()
data_updater = DataUpdateService(time_frame_unit=TimeFrameUnit.DAY)

#symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS50')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS100')
symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS200')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS500')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('NAS100')
ticker_list = [x.symbol for x in symbol_group]
# print (ticker_list)
ticker_list = [
    'MOD', 'DAKT', 'CRDO', 'ADMA', 'PLTR', 'MSTR', 'SMCI', 'CCL', 'DFH', 'INOD', 'VIST',
    'AMRX', 'CLS', 'GGAL', 'UWMC', 'VKTX', 'AMSC', 'CAMT', 'COIN', 'CVNA', 'NU', 'AAOI',
    'AVPT', 'FIP', 'GATO', 'MRUS', 'NVDA', 'TDW', 'BBAR', 'BMA', 'CABA', 'MMYT', 'NUVL',
    'OII', 'ORN', 'REAX', 'SPOT', 'SUPV', 'TK', 'VST', 'ZETA', 'AVGO', 'CLBT', 'FTAI',
    'HCC', 'HLX', 'HOOD', 'IOT', 'MAMA', 'ONON', 'OSCR', 'RKT', 'ROIV', 'TGLS', 'UBER',
    'AGI', 'ANF', 'BRBR', 'EGO', 'FTI', 'GSL', 'HMY', 'HRTG', 'MTH', 'MUX', 'ONTO', 'PHM',
    'RCL', 'RDNT', 'THC', 'VRT', 'XPO', 'AMD', 'APO', 'BV', 'CUK', 'CVLT', 'GCT', 'LFMD',
    'LRCX', 'ORCL', 'PWP', 'SCCO', 'SMMT', 'ACGL', 'ALKT', 'AROC', 'ASC', 'AXON', 'BLDR',
    'BX', 'DOLE', 'HUT', 'IREN', 'KKR', 'NXE', 'ROAD', 'TOL', 'WPM', 'ACLS'
]

start_date: datetime = datetime(year=2023, month=1, day=1)
end_date: datetime = datetime(year=2024, month=8, day=30)

for ticker in ticker_list:
    if ticker not in ['AVGO']:
        count = data_updater.darvas_box_strategy.trading_signals_count(ticker, start_date, end_date,)
        # print (ticker, start_date, end_date)
        if count > 0:
            df = data_updater.darvas_box_strategy.df.copy()
            # print (ticker, len(df), df.info)
            df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
            df = df.set_index(["hdate"])

            bt = Backtest(df, SignalStrategy, cash = 10000)
            stats=bt.run()
            # print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean(), len(stats._trades))

            if not stats._trades.empty:
                # stats._trades["ticker"] = ticker
                df_collect = pd.concat([df_collect, stats._trades], axis=0)
if len(df_collect) > 0:
    print(df_collect["PnL"].mean(), df_collect["Duration"].mean(), len(df_collect))
else:
    print("Empty collection")

# print(df_collect.info())

fig = go.Figure()

colors = np.where(df_collect['PnL'] >= 0, 'blue', 'red')
# Commented out long hover text line to avoid line length issues
#hover_text = [
#    f"Ticker: {row['ticker']}<br>PnL: {row['PnL']:.2f}<br>"
#    f"EntryPrice: {row['EntryPrice']:.2f}<br>ExitPrice: {row['ExitPrice']:.2f}<br>"
#    f"EntryTime: {row['EntryTime']}<br>ExitTime: {row['ExitTime']}"
#    for _, row in df_collect.iterrows()
#]

#fig.add_trace(go.Scatter(
#    x=df_collect['ExitTime'],  # x-axis: ExitTime
#    y=df_collect['PnL'],       # y-axis: PnL
#    mode='markers',       # Display points only
#    marker=dict(size=8, color=colors),  # Customize the marker
#    text=hover_text,  # Add the hover text
#    hoverinfo='text',  # Show only the custom hover text
#    name='PnL per Trade'
#))

# Update layout for better appearance
fig.update_layout(
    title='PnL vs Exit Time',
    xaxis_title='Exit Time',
    yaxis_title='PnL',
    xaxis={'tickformat': '%Y-%m-%d', 'showgrid': True},  # Format date and time properly
    yaxis={'showgrid': True},
    width=1000,  # Set the width of the plot
    height=600   # Set the height of the plot
)

fig.show()
# pio.renderers.default = 'browser'
# pio.show(fig)

In [3]:
class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        current_signal = self.data.buy_signal[-1]
        if current_signal and not self.position:
            self.buy(sl=self.data.hard_stoploss)
        # elif self.position and self.data.Close[-1] < self.data.ema_20[-1]:
        elif self.data.macd_histogram[-1] < 0:
            self.position.close()

ticker = 'AMZN'
start_date: datetime = datetime(year=2017, month=1, day=1)
end_date: datetime = datetime(year=2024, month=10, day=1)

data_updater = DataUpdateService(time_frame_unit=TimeFrameUnit.WEEK)
data_updater.mars_strategy.calculate_entries(ticker,start_date, end_date,)
count = data_updater.mars_strategy.df["buy_signal"].sum()
if count > 0:
    df = data_updater.mars_strategy.df.copy()
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    df = df.set_index(["hdate"])
    bt = Backtest(df, SignalStrategy, cash = 10000)
    stats=bt.run()
    # print(stats)

    trades = stats._trades
    trades["trade_duration"] = trades['Duration'].dt.days
    trades["trade_duration"] = trades["trade_duration"].replace(0,5)
    trades["cagr"] = (trades["ExitPrice"] / trades["EntryPrice"]) ** (365 / trades["trade_duration"] ) - 1
    # print(trades)

    print(f"CAGR - {trades["cagr"].mean()}")
    print(f"ROI  - {trades["PnL"].sum()/70000}")

CAGR - 0.1971143405570513
ROI  - 0.17522357142857137


In [5]:
df[300:355]

,Open,High,Low,Close,Volume,trade_count,max_box_4,min_box_4,consolidation_change,hard_stoploss,ema_10,ema_20,macd,macd_histogram,macd_signal,max_close_10,ema_volume_4,volume_change,buy_signal
hdate,,,,,,,,,,,,,,,,,,,
2022-10-02,113.295,118.7000,112.8400,113.000,327443910,2799945,134.095,113.780,0.179779,123.9175,129.9370,121.89300,-4.806103,0.142209,-4.948312,143.55,2.942355e+08,1.112863,False
2022-10-09,113.580,123.0000,112.4500,114.560,282117906,2463001,134.095,113.000,0.184139,123.5275,127.8980,122.24150,-5.239719,-0.233125,-5.006593,143.55,3.041932e+08,0.927430,False
2022-10-16,115.100,116.2500,105.3450,106.900,320279109,2983449,134.095,113.000,0.197334,123.5275,124.5080,121.82900,-6.130788,-0.899356,-5.231432,143.55,3.240973e+08,0.988219,False
2022-10-23,110.110,119.5900,110.0900,119.320,305358168,2498728,122.160,106.900,0.127891,114.5100,122.0850,121.67750,-5.768284,-0.429481,-5.338803,138.23,3.060572e+08,0.997716,False
2022-10-30,119.980,121.3152,97.6600,103.410,555239926,4805358,119.320,106.900,0.120104,113.0900,118.6030,121.36550,-6.687709,-1.079125,-5.608584,133.27,3.087998e+08,1.798058,False
2022-11-06,103.560,104.8700,88.0400,90.980,686094076,5526083,119.980,103.410,0.182128,111.6750,114.6260,120.60350,-8.323410,-2.171861,-6.151549,133.27,3.657488e+08,1.875862,False
2022-11-13,91.950,101.1900,85.8700,100.790,580197472,4379146,119.980,90.980,0.287727,105.4600,111.9540,119.82000,-8.727524,-2.060780,-6.666744,133.27,4.667428e+08,1.243077,False
2022-11-20,98.770,103.7900,92.4800,94.140,486974690,3422832,119.980,90.980,0.308052,105.4600,108.0410,119.04900,-9.475163,-2.246735,-7.228428,123.53,5.317224e+08,0.915844,False
2022-11-27,93.970,95.0200,90.5900,93.410,259384496,1865201,119.980,90.980,0.310459,105.4600,105.0290,117.94250,-10.011175,-2.226198,-7.784977,119.32,5.771265e+08,0.449441,False


In [6]:
class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        current_signal = self.data.buy_signal[-1]
        if current_signal and not self.position:
            # and self.data.Open[0] > self.data.hard_stoploss[0]:
            # print(f"Open: {self.data.Open[0]} SL: {self.data.hard_stoploss[0]}")
            self.buy(sl=self.data.hard_stoploss)
        # elif self.position and self.data.Close[-1] < self.data.ema_20[-1]:
        elif self.data.macd_histogram[-1] < 0:
            self.position.close()

# ticker = 'AMZN'
start_date: datetime = datetime(year=2022, month=1, day=1)
end_date: datetime = datetime(year=2023, month=12, day=31)
data_updater = DataUpdateService(time_frame_unit=TimeFrameUnit.WEEK)
group_trades = pd.DataFrame()

for symbol_group in data_updater.get_symbol_group_list("NAS100"):
    ticker = symbol_group.symbol
    # if not ticker == 'QCOM':
    #     continue
    # print(ticker)
    data_updater.mars_strategy.calculate_entries(ticker,start_date, end_date,)
    if (not data_updater.mars_strategy.df.empty and "buy_signal" in data_updater.mars_strategy.df.columns) :
        if data_updater.mars_strategy.df["buy_signal"].sum() > 0 :
            df = data_updater.mars_strategy.df.copy()
            df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
            df = df.set_index(["hdate"])
            bt = Backtest(df, SignalStrategy, cash = 10000)
            stats=bt.run()

            if not stats._trades.empty:
                trades = stats._trades
                trades["trade_duration"] = trades['Duration'].dt.days
                trades["trade_duration"] = trades["trade_duration"].replace(0,5)
                trades["cagr"] = (trades["ExitPrice"] / trades["EntryPrice"]) ** (365 / trades["trade_duration"] ) - 1
                trades["symbol"] = ticker

                group_trades = pd.concat([group_trades, trades], ignore_index=True)
            # print(trades)

print(f"CAGR - {group_trades["cagr"].mean()}")
print(f"ROI  - {group_trades["PnL"].sum()}")

/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


CAGR - 0.04995037777487873
ROI  - 44588.12699999998


In [7]:
pd.set_option('display.max_rows', None)
print(group_trades.groupby('symbol')['PnL'].sum())
print(group_trades['PnL'].sum()/100)
# df[70:80]

symbol
AAPL     1551.8000
ABNB     -241.6800
ADBE     2042.4150
ADI      -386.5350
ADP       335.5500
ADSK    -1673.7400
AEP      -371.9900
AMAT     -446.7750
AMD      6463.2900
AMGN    -1722.5400
AMZN     2678.2800
ANSS      527.0200
ASML      371.7800
AVGO     6850.5600
AZN     -1809.1800
BIIB     -840.9400
BKNG     1750.1450
BKR       630.5800
CCEP      195.7100
CDNS     1379.8600
CDW      -574.6050
CEG      1540.0400
CHTR     -874.9200
CMCSA    -129.3300
COST     1227.3150
CPRT     2792.0100
CRWD     3632.9775
CSCO     -378.9400
CSGP    -1416.5250
CSX      -706.4900
CTAS      589.4400
CTSH    -1129.7000
DASH     1325.7700
DDOG    -1245.7900
DLTR    -1180.7550
DXCM    -1972.3850
EA        -63.6900
EXC      -732.3450
FANG     -630.8650
FAST     1709.5400
FTNT     -163.1800
GEHC      359.1000
GFS      -555.1200
GILD     3049.4200
GOOG     1199.8600
GOOGL    2346.5000
HON      -215.2000
IDXX     -304.7400
ILMN    -1037.2500
INTC     1601.0100
INTU     1318.1200
ISRG    -1146.5900
KDP  

In [8]:
# print(data_updater.mars_strategy.df)
print(group_trades.query("symbol == 'TSLA'"))

     Size  EntryBar  ExitBar  EntryPrice  ExitPrice     PnL  ReturnPct  \
289    33        37       37     300.090    285.980 -465.63  -0.047019   
290    33        80       80     286.625    262.195 -806.19  -0.085233   

     EntryTime   ExitTime Duration  trade_duration      cagr symbol  
289 2022-09-19 2022-09-19   0 days               5 -0.970274   TSLA  
290 2023-07-17 2023-07-17   0 days               5 -0.998502   TSLA  
